In [62]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from scipy.stats import zscore
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normaliser
from sklearn.svm import SVC
Normaliser = Normaliser()



pd.set_option('display.max_columns', None)


#path = input("Enter CSV file: ")
path = "CTU-IoT-Malware-Capture-9-1conn.log.labeled.csv"

print(path)

CTU-IoT-Malware-Capture-9-1conn.log.labeled.csv


In [73]:
cell_df = pd.read_csv(path, delimiter="|")


#cell_df = pd.read_csv(path, delimiter="|", nrows=10000)
cell_df=cell_df.replace('-', None)
cell_df.head()





,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed-label
985924,1.532535e+09,Cmyh7a2ocpMm0NQVRb,192.168.100.111,22127.0,114.80.50.91,23.0,tcp,None,None,None,None,S0,None,None,0.0,S,1.0,40.0,0.0,0.0,None,Malicious,PartOfAHorizontalPortScan
2509935,1.532552e+09,CPN2Dr33yRcGTx9OLb,192.168.100.111,6314.0,59.6.167.124,81.0,tcp,None,None,None,None,S0,None,None,0.0,S,1.0,40.0,0.0,0.0,None,Malicious,PartOfAHorizontalPortScan
3982712,1.532567e+09,CKLCDs4UqyhbwENzK7,192.168.100.111,60086.0,88.134.199.17,23.0,tcp,None,None,None,None,S0,None,None,0.0,S,1.0,40.0,0.0,0.0,None,Malicious,PartOfAHorizontalPortScan
3723977,1.532564e+09,C1jRhz1wuGLBjCGZDi,192.168.100.111,30256.0,183.39.227.210,81.0,tcp,None,None,None,None,S0,None,None,0.0,S,1.0,40.0,0.0,0.0,None,Malicious,PartOfAHorizontalPortScan
3972911,1.532567e+09,CeD0DL1mlWRKoIpEq1,192.168.100.111,56981.0,84.98.173.210,23.0,tcp,None,None,None,None,S0,None,None,0.0,S,1.0,40.0,0.0,0.0,None,Malicious,PartOfAHorizontalPortScan


In [80]:
print(cell_df.columns)

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'conn_state', 'missed_bytes', 'history', 'orig_pkts',
       'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label',
       'detailed-label'],
      dtype='object')


In [ ]:
columns_to_remove = []
def calculateNull():
    for column in cell_df.columns:
        null_values = (cell_df[column] == '-').sum()
        total_values = len(cell_df[column])  # Count total number of rows
        difference = (null_values / total_values) * 100
    
        
        print("Column:", column, "Nulls:", null_values, "Total:", total_values, "(", difference, "%)")
        columns_to_remove.append(column)
    
    
    print(columns_to_remove)


def newNull():
    for column in cell_df.columns:
        null_values = cell_df[column].isnull().sum()
        total_values = len(cell_df[column])  # Count total number of rows
        difference = (null_values / total_values) * 100
        print("Column:", column, "Nulls:", null_values, "Total:", total_values, "(", difference, "%)")
        if difference > 60:
            columns_to_remove.append(column)


newNull()
cell_df = cell_df.drop(columns=columns_to_remove)



In [83]:



for column in cell_df.columns:
    unique_count = cell_df[column].nunique()
    if unique_count < 60:
        print(column, unique_count, cell_df[column].unique())
        #print(cell_df[column].value_counts())
    else:
        print(column, unique_count)




ts 200000
uid 200000
id.orig_h 89
id.orig_p 62471
id.resp_h 191096
id.resp_p 317
proto 3 ['tcp' 'udp' 'icmp']
conn_state 7 ['S0' 'OTH' 'SF' 'REJ' 'RSTRH' 'RSTO' 'SH']
missed_bytes 1 [0.]
history 12 ['S' 'D' None 'ShADadFf' 'ShAdDafF' 'Sr' 'Dd' '^r' 'ShR' 'ShAdDaR'
 'ShADdFaf' 'Fa' 'ShAdDaFfR']
orig_pkts 23 [ 1.  2.  7.  3. 21.  9.  8.  6.  4. 36. 10.  0. 12. 45. 13. 31.  5. 24.
 19. 23. 30. 15. 11.]
orig_ip_bytes 86
resp_pkts 10 [ 0.  5. 20.  1.  2.  3. 29.  4. 26. 17.]
resp_ip_bytes 38 [   0.  927. 1263.   40.  330.  680.   83. 1007.   81. 1744.  340.  322.
   86.  313.   98.  334.  347.   44. 9156.  905.   89. 1692.  327.  296.
 1388.  345.   77. 1325.  516.   76.  356.  317.  390.  328. 1235.   96.
  981. 1113.]
label 2 ['Malicious' 'Benign']
detailed-label 1 ['PartOfAHorizontalPortScan' None]


In [84]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

#class_labels = cell_df['class']
#protocol_labels = cell_df['protocol_type']
#flag_labels = cell_df['flag']
#service_labels = cell_df['service']

def encode_feature(column):
    return label_encoder.fit_transform(column)

for column in cell_df.columns:
    # If not numerical, then encode.
    if not pd.api.types.is_numeric_dtype(cell_df[column]):
        # If the data type is numerical, apply the function 'fun'
        cell_df[column] = encode_feature(cell_df[column])

cell_df.head()


,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,detailed-label
985924,1.532535e+09,158078,37,22127.0,12562,23.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0
2509935,1.532552e+09,82065,37,6314.0,157564,81.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0
3982712,1.532567e+09,65787,37,60086.0,180862,23.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0
3723977,1.532564e+09,5635,37,30256.0,99588,81.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0
3972911,1.532567e+09,129723,37,56981.0,178314,23.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0


In [85]:
# Data Preprocessing
# We need to convert all categorical features into numerical.

# Used to convert categorical features into numerical 0 or 1.
from sklearn.preprocessing import LabelEncoder
#label_encoder = LabelEncoder()

#encoded_class_labels = label_encoder.fit_transform(class_labels)
#encoded_protocol_labels = label_encoder.fit_transform(protocol_labels)
#encoded_flag_labels = label_encoder.fit_transform(flag_labels)
#encoded_service_labels = label_encoder.fit_transform(service_labels)


#cell_df['class'] = encoded_class_labels
#cell_df['protocol_type'] = encoded_protocol_labels
#cell_df['flag'] = encoded_flag_labels
#cell_df['service'] = encoded_service_labels

cell_df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,detailed-label
985924,1.532535e+09,158078,37,22127.0,12562,23.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0
2509935,1.532552e+09,82065,37,6314.0,157564,81.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0
3982712,1.532567e+09,65787,37,60086.0,180862,23.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0
3723977,1.532564e+09,5635,37,30256.0,99588,81.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0
3972911,1.532567e+09,129723,37,56981.0,178314,23.0,1,4,0.0,3,1.0,40.0,0.0,0.0,1,0


In [90]:
X = cell_df.drop('label', axis=1)
y = cell_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

label
1    159430
0       570
Name: count, dtype: int64

In [10]:

X = cell_df.drop(['label'], axis='columns')
X_normalized = normaliser.fit_transform(X)
print(X_normalized)

[[1.87685969e-03 1.00000000e+00 9.00000000e-01 ... 5.08531281e-03
  8.77192982e-03 5.78492266e-04]
 [3.35698482e-04 5.00000000e-01 0.00000000e+00 ... 2.22147876e-02
  1.75438596e-02 1.52234807e-03]
 [3.35698482e-04 5.00000000e-01 6.00000000e-01 ... 1.26463700e-02
  0.00000000e+00 0.00000000e+00]
 ...
 [3.50957504e-04 5.00000000e-01 7.00000000e-01 ... 1.47206424e-02
  8.77192982e-03 3.34916575e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 4.55001673e-03
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.74707260e-03
  0.00000000e+00 0.00000000e+00]]


In [18]:
model = SVC(kernel="linear")

In [19]:
# ~ 3 minutes
model.fit(X_train, y_train)

SVC(kernel='linear')

In [1]:
model.score(X_test, y_test)

NameError: name 'model' is not defined

In [ ]:
Accuracy: 66.87%